# Notebook 03: Exploratory Data Analysis

The EDA conducted here falls into the following 5 steps:
1. Summary Data for Nikaya contents
2. Creating Full DataFrame with Nikaya Information
3. Compiling 'See Also' Translator Recommendations
4. Compiling per Sutta Recommendations
5. Adding all Relevant Information to the Primary DF
6. Countvectorization for Each Nikaya

The dataframes that were created were used to create two visualizations. This information was then used to create sutta by sutta [visualizations](https://public.tableau.com/profile/alec.johnson2268#!/vizhome/SuttaInformation/SuttaView) as well as full collection [visualizations](https://public.tableau.com/profile/alec.johnson2268#!/vizhome/FullSuttaVizualizations/ThePaliCanon) in Tableau.

In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv('./sutta_csv/cleaned/df_all_clean_2.csv')

In [3]:
df.shape

(1306, 9)

### Step 1: Summary Data - Nikaya

In [4]:
df.groupby('nikaya').count()

,title,ref,title_url,summary,text_full,intro,notes,see_also
nikaya,,,,,,,,
Anguttara Nikaya,385,385,385,385,385,385,385,385
Digha Nikaya,14,14,14,14,14,14,13,14
Khuddaka Nikaya,427,427,427,427,427,427,427,427
Majjhima Nikaya,101,101,101,101,101,101,101,101
Samyutta Nikaya,379,379,379,379,379,379,379,379


In [5]:
# Making word counts

df['word_count'] = df['text_full'].str.count(' ') + 1

mean_words = df.groupby('nikaya')[['word_count']].mean()

mean_words = mean_words.rename(columns = {
                                 'word_count': 'mean_words'})

In [6]:
#Total sum of words

sum_words = df.groupby('nikaya')[['word_count']].sum()

sum_words = sum_words.rename(columns = {
                                 'word_count': 'sum_words'})


In [7]:
sum_mean = pd.merge(sum_words, mean_words, on = 'nikaya')

In [8]:
# Assuming 250 words per page and 265 WPM reading speed

sum_mean['total_pages'] = sum_mean['sum_words'] / 250
sum_mean['average_pages'] = sum_mean['mean_words'] / 250
sum_mean['total_read_time_hrs'] = sum_mean['sum_words'] / (265 * 60)
sum_mean['average_read_time_mins'] = sum_mean['mean_words'] / 265

In [9]:
sum_mean

,sum_words,mean_words,total_pages,average_pages,total_read_time_hrs,average_read_time_mins
nikaya,,,,,,
Anguttara Nikaya,296039,768.932468,1184.156,3.075730,18.618805,2.901632
Digha Nikaya,141063,10075.928571,564.252,40.303714,8.871887,38.022372
Khuddaka Nikaya,415757,973.669789,1663.028,3.894679,26.148239,3.674226
Majjhima Nikaya,319603,3164.386139,1278.412,12.657545,20.100818,11.941080
Samyutta Nikaya,265004,699.218997,1060.016,2.796876,16.666918,2.638562


In [10]:
full_canon = sum_mean.sum()
full_canon.name = 'Full Canon'
sum_mean = sum_mean.append(full_canon.T)

In [11]:
sum_mean.reset_index(inplace = True)

### Step 2: DataFrame Summary Information Nikaya

#### Summary of each Nikaya

In [12]:
sutta_names = {'Majjhima Nikaya': [101], 'Samyutta Nikaya': [379], 'Digha Nikaya': [14], 'Anguttara Nikaya': [385], 'Khuddaka Nikaya': [427], 'Full Canon': [1306]}

In [13]:
df_eda = pd.DataFrame(sutta_names).T

df_eda.reset_index(inplace=True)

df_eda = df_eda.rename(columns = {0: "count",
                                 'index': 'nikaya'})

In [14]:
df_eda = pd.merge(df_eda, sum_mean, on = 'nikaya')

In [15]:
df_eda

,nikaya,count,sum_words,mean_words,total_pages,average_pages,total_read_time_hrs,average_read_time_mins
0,Majjhima Nikaya,101,319603.0,3164.386139,1278.412,12.657545,20.100818,11.941080
1,Samyutta Nikaya,379,265004.0,699.218997,1060.016,2.796876,16.666918,2.638562
2,Digha Nikaya,14,141063.0,10075.928571,564.252,40.303714,8.871887,38.022372
3,Anguttara Nikaya,385,296039.0,768.932468,1184.156,3.075730,18.618805,2.901632
4,Khuddaka Nikaya,427,415757.0,973.669789,1663.028,3.894679,26.148239,3.674226
5,Full Canon,1306,1437466.0,15682.135964,5749.864,62.728544,90.406667,59.177872


In [16]:
# Getting means where sums are currently

df_eda.at[5,'mean_words'] = df_eda.at[5,'mean_words'] / 5
df_eda.at[5,'average_pages'] = df_eda.at[5,'average_pages'] / 5
df_eda.at[5,'average_read_time_mins'] = df_eda.at[5,'average_read_time_mins'] / 5

In [18]:
# df_eda.to_csv('./sutta_csv/eda/df_eda.csv', index = False)

### Step 3: Compiling 'See Also' Recommendations

In [19]:
## Drop all with no 'see also' data

# df = df[~df.see_also.str.contains("No Additional")]

In [22]:
## Removing First Colon and \xa0 in between values

# df['see_also_cl'] = df['see_also'].str.replace(': ', '').copy()

# df['see_also_cl'] = df['see_also_cl'].str.replace('\xa0', ' ').copy()

#df['see_also_cl'] = df['see_also_cl'].str.replace('\ ', ' ').copy()

In [23]:
## Splitting at each semicolon
# df['see_also_cl'] = df['see_also_cl'].str.split(';')

In [24]:
## Appending all to one list

# all_see = []
# for i in df['see_also_cl']:
#     for j in i:
#         all_see.append(j.strip())

In [25]:
## Creating per-Nikaya lists

# MN = []
# SN = []
# DN = []
# AN = []
# KN = []

# for i in all_see:
#     if i.__contains__('MN'):
#         MN.append(i)
#     elif i.__contains__('SN'):
#         SN.append(i)
#     elif i.__contains__('DN'):
#         DN.append(i)
#     elif i.__contains__('AN'):
#         AN.append(i)
#     else: 
#         KN.append(i)

In [3]:
## Function to count occurence of recommendations and put in a per-Nikaya dataframe

# def count_df(sutta_list):
#     sutta_counts = {x: [sutta_list.count(x)] for x in sutta_list}
#     df_sc = pd.DataFrame(sutta_counts).T
#     df_sc.reset_index(inplace=True)
#     df_sc = df_sc.rename(columns = {"index": "sutta"})
    
#     return df_sc

In [27]:
## Making and concatenating all dataframes

# df_mn = count_df(MN)
# df_mn = df_mn.rename(columns = {0: "count"})
# df_mn.sort_values(by = 'count', inplace = True, ascending= False)
# df_mn['ref'] = 'MN'

# df_sn = count_df(SN)
# df_sn = df_sn.rename(columns = {0: "count"})
# df_sn.sort_values(by = 'count', inplace = True, ascending= False)
# df_sn['ref'] = 'SN'

# df_dn = count_df(DN)
# df_dn = df_dn.rename(columns = {0: "count"})
# df_dn.sort_values(by = 'count', inplace = True, ascending= False)
# df_dn['ref'] = 'DN'

# df_an = count_df(AN)
# df_an = df_an.rename(columns = {0: "count"})
# df_an.sort_values(by = 'count', inplace = True, ascending= False)
# df_an['ref'] = 'AN'

# df_kn = count_df(KN)
# df_kn = df_kn.rename(columns = {0: "count"})
# df_kn.sort_values(by = 'count', inplace = True, ascending= False)
# df_kn['ref'] = 'KN'

In [28]:
# df_all_count = pd.concat([df_mn, df_sn, df_dn, df_an, df_kn], axis=0)

In [29]:
#df_all_count.to_csv('./sutta_csv/eda/df_all_counts.csv', index = False)

### Step 4: Compiling per Sutta Recommendations

Now taking in the recommendations from the recommendation engine for each reference

In [30]:
def most_similar(ref):
    '''
  doc_id: (str) unique document identifier
  similarity_matrix: (ndarray) cosine similarity or euclidean difference matrix
  matrix_type: (str) 'cosine' or 'euclidean'
  number: (int) n of similar suttas to display
    '''
    df = pd.read_csv('./sutta_csv/cleaned/df_all_prep.csv')
    df['ref'] = df['ref'].str.replace('\u2009', ' ')
    
    ref = ref.replace('\u2009', ' ')

    with open('./data/pairwise_similarities.npy', 'rb') as f:
        pairwise_similarities = np.load(f)
    
    def get_ix(ref):
        return df[df['ref'] == ref].index[0]

    doc_ix = get_ix(ref)
    
    similar_ix = np.argsort(pairwise_similarities[doc_ix])[::-1][1:]
    
    rec_list = []
    for ix in similar_ix[1:6]:
        rec_list.append(df.iloc[ix]["ref"])
            

    return rec_list

In [31]:
df2 = pd.read_csv('./sutta_csv/cleaned/df_all_clean_2.csv')

In [32]:
# # List of all references to be fed

# refs = []
# for i in df2['ref']:
#     refs.append(i)


In [33]:
# recs = []
# for i in refs:
#     recs.append(most_similar(i))

In [34]:
# #all in one list

# all_recs = []
# for i in recs:
#     for j in i:
#         all_recs.append(j)

In [35]:
## Nikaya by Nikaya

# MN = []
# SN = []
# DN = []
# AN = []
# KN = []

# for i in all_recs:
#     if i.__contains__('MN'):
#         MN.append(i)
#     elif i.__contains__('SN'):
#         SN.append(i)
#     elif i.__contains__('DN'):
#         DN.append(i)
#     elif i.__contains__('AN'):
#         AN.append(i)
#     else: 
#         KN.append(i)

In [36]:
## Creating and concatenating each nikaya

# df_mnr = count_df(MN)
# df_mnr = df_mnr.rename(columns = {0: "count"})
# df_mnr.sort_values(by = 'count', inplace = True, ascending= False)
# df_mnr['ref'] = 'MN'

# df_snr = count_df(SN)
# df_snr = df_snr.rename(columns = {0: "count"})
# df_snr.sort_values(by = 'count', inplace = True, ascending= False)
# df_snr['ref'] = 'SN'

# df_dnr = count_df(DN)
# df_dnr = df_dnr.rename(columns = {0: "count"})
# df_dnr.sort_values(by = 'count', inplace = True, ascending= False)
# df_dnr['ref'] = 'DN'

# df_anr = count_df(AN)
# df_anr = df_anr.rename(columns = {0: "count"})
# df_anr.sort_values(by = 'count', inplace = True, ascending= False)
# df_anr['ref'] = 'AN'

# df_knr = count_df(KN)
# df_knr = df_knr.rename(columns = {0: "count"})
# df_knr.sort_values(by = 'count', inplace = True, ascending= False)
# df_knr['ref'] = 'KN'

In [37]:
# df_rec_count = pd.concat([df_mnr, df_snr, df_dnr, df_anr, df_knr], axis=0)

In [38]:
# df_rec_count.head()

In [39]:
#df_rec_count.to_csv('./sutta_csv/eda/df_rec_counts.csv', index = False)

### Step 5: Adding all Relevant Information to the Primary DF

In [40]:
## Reading in secondary dfs
df_recs = pd.read_csv('./sutta_csv/eda/df_rec_counts.csv')
df_salso = pd.read_csv('./sutta_csv/eda/df_all_counts.csv')

#### Changes to secondary dfs

In [41]:
df_recs.drop(columns = 'ref', inplace = True)
df_recs = df_recs.rename(columns = {
                                 'sutta': 'ref',
                                'count': 'count_recs'})

In [42]:
df_salso.drop(columns = 'ref', inplace = True)
df_salso = df_salso.rename(columns = {
                                 'sutta': 'ref',
                                'count': 'count_see_also'})

#### Changes to primary DF and Merging

In [43]:
df2['see_also'] = df2['see_also'].str.replace(': ', '').copy()

df2['ref'] = df2['ref'].str.replace('\u2009', ' ').copy()

In [44]:
df2 = pd.merge(right = df2, left = df_recs, on = 'ref', how = 'right')
df2 = df2.fillna('None')
df2 = pd.merge(right = df2, left = df_salso, on = 'ref', how = 'right')
df2 = df2.fillna('None')

In [45]:
df2.head()

ref count_see_also count_recs  \
0  MN 1             12          4   
1  MN 2             16         23   
2  MN 4             12         14   
3  MN 5              2          9   
4  MN 6              4         10   

                                          title  \
0  MN 1  Mūlapariyāya Sutta | The Root Sequence   
1     MN 2  Sabbāsava Sutta | All the Effluents   
2     MN 4  Bhaya-bherava Sutta | Fear & Terror   
3            MN 5  Anaṅgaṇa Sutta | Unblemished   
4    MN 6  Ākaṅkheyya Sutta | If One Would Wish   

                                        title_url  \
0  https://www.dhammatalks.org/suttas/MN/MN1.html   
1  https://www.dhammatalks.org/suttas/MN/MN2.html   
2  https://www.dhammatalks.org/suttas/MN/MN4.html   
3  https://www.dhammatalks.org/suttas/MN/MN5.html   
4  https://www.dhammatalks.org/suttas/MN/MN6.html   

                                                                                                                                                                                                                                                                                                    summary  \
0  MN 1  Mūlapariyāya Sutta | The Root Sequence  —  The Majjhima Nikāya opens with one of the few suttas where his listeners did NOT delight in his words. In it, the Buddha dismisses the tendency—common both in his time and in ours—to posit a metaphysical principle from which the universe emanates.   
1                                                                                                     MN 2  Sabbāsava Sutta | All the Effluents  —  The Buddha lists seven approaches for eliminating the āsavas, or effluents: deep-seated defilements that “flow out” of the mind and prevent liberation.   
2                                                                                                                                                                                  MN 4  Bhaya-bherava Sutta | Fear & Terror  —  The qualities of mind necessary for living in the wilderness without fear.   
3                                                                                                                                                                      MN 5  Anaṅgaṇa Sutta | Unblemished  —  Ven. Sāriputta explains the blemishes of the mind: the influences of evil, unskillful wishes.   
4                                              MN 6  Ākaṅkheyya Sutta | If One Would Wish  —  The wishes that can be fulfilled by brining the precepts to perfection, being committed to inner tranquility of awareness, not neglecting jhāna, being endowed with insight, and frequenting empty dwellings.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

#### Adding relevant columns for reading times, recs and numbers of words

In [46]:
df2['word_count'] = df2['text_full'].str.count(' ') + 1

In [47]:
# Assuming 250 words per page and 265 WPM reading speed

df2['pages'] = round(df2['word_count'] / 250, 1)
df2['read_time_mins'] = round(df2['word_count'] / 265)

In [48]:
df2.head()

ref count_see_also count_recs  \
0  MN 1             12          4   
1  MN 2             16         23   
2  MN 4             12         14   
3  MN 5              2          9   
4  MN 6              4         10   

                                          title  \
0  MN 1  Mūlapariyāya Sutta | The Root Sequence   
1     MN 2  Sabbāsava Sutta | All the Effluents   
2     MN 4  Bhaya-bherava Sutta | Fear & Terror   
3            MN 5  Anaṅgaṇa Sutta | Unblemished   
4    MN 6  Ākaṅkheyya Sutta | If One Would Wish   

                                        title_url  \
0  https://www.dhammatalks.org/suttas/MN/MN1.html   
1  https://www.dhammatalks.org/suttas/MN/MN2.html   
2  https://www.dhammatalks.org/suttas/MN/MN4.html   
3  https://www.dhammatalks.org/suttas/MN/MN5.html   
4  https://www.dhammatalks.org/suttas/MN/MN6.html   

                                                                                                                                                                                                                                                                                                    summary  \
0  MN 1  Mūlapariyāya Sutta | The Root Sequence  —  The Majjhima Nikāya opens with one of the few suttas where his listeners did NOT delight in his words. In it, the Buddha dismisses the tendency—common both in his time and in ours—to posit a metaphysical principle from which the universe emanates.   
1                                                                                                     MN 2  Sabbāsava Sutta | All the Effluents  —  The Buddha lists seven approaches for eliminating the āsavas, or effluents: deep-seated defilements that “flow out” of the mind and prevent liberation.   
2                                                                                                                                                                                  MN 4  Bhaya-bherava Sutta | Fear & Terror  —  The qualities of mind necessary for living in the wilderness without fear.   
3                                                                                                                                                                      MN 5  Anaṅgaṇa Sutta | Unblemished  —  Ven. Sāriputta explains the blemishes of the mind: the influences of evil, unskillful wishes.   
4                                              MN 6  Ākaṅkheyya Sutta | If One Would Wish  —  The wishes that can be fulfilled by brining the precepts to perfection, being committed to inner tranquility of awareness, not neglecting jhāna, being endowed with insight, and frequenting empty dwellings.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [49]:
# List of all references to be fed

# refs = []
# for i in df2['ref']:
#     refs.append(i)

In [50]:
# df2['recs'] = df2.apply(lambda x : most_similar(x['ref']), axis = 1)

In [51]:
# df2.to_csv('./sutta_csv/eda/df_full_info.csv', index = False)

### Step 6: Countvectorization for Each Nikaya

In [52]:
my_stopwords2 = list(CountVectorizer(stop_words = 'english').get_stop_words()) + ['just', 
                                                                                  'like', 've', 'don','really', 'things', 'deleted', 'feel', 'want',
                                                                                  'years', 'help', 'going', 'make', 'day', 'having', 'bad', 'got', 'way', 
                                                                                  "I'm", "I've", "I'll", 'n', 't', 's', 'm', 'g', 'll', 'doesn']

In [54]:
df2.head(0)

,ref,count_see_also,count_recs,title,title_url,summary,text_full,intro,notes,see_also,nikaya,word_count,pages,read_time_mins


In [57]:
#Splitting into different dfs per nikaya

df_mn = df2.loc[df2['nikaya'] == 'Majjhima Nikaya']
df_sn = df2.loc[df2['nikaya'] == 'Samyutta Nikaya']
df_dn = df2.loc[df2['nikaya'] == 'Digha Nikaya']
df_an = df2.loc[df2['nikaya'] == 'Anguttara Nikaya']
df_kn = df2.loc[df2['nikaya'] == 'Khuddaka Nikaya']

In [59]:
#Function for countvectorization

def countvecr(df):
    cvec = CountVectorizer(stop_words = my_stopwords2)

    cvec.fit(df['text_full'])

    z = cvec.transform(df2['text_full'])
    z_df = pd.DataFrame(z.todense(), columns = cvec.get_feature_names())
    z_df['ref'] = df['ref']
    z_df['nikaya'] = df['nikaya']
    return z_df

In [60]:
# Calling on each nikaya df

df_mnvec = countvecr(df_mn)
df_snvec = countvecr(df_sn)
df_dnvec = countvecr(df_dn)
df_anvec = countvecr(df_an)
df_knvec = countvecr(df_kn)

In [61]:
##Reading out

# df_mnvec.to_csv('./sutta_csv/cleaned/Tableau/df_mnvec.csv', index = False)
# df_snvec.to_csv('./sutta_csv/cleaned/Tableau/df_snvec.csv', index = False)
# df_dnvec.to_csv('./sutta_csv/cleaned/Tableau/df_dnvec.csv', index = False)
# df_anvec.to_csv('./sutta_csv/cleaned/Tableau/df_anvec.csv', index = False)
# df_knvec.to_csv('./sutta_csv/cleaned/Tableau/df_knvec.csv', index = False)